<a href="https://colab.research.google.com/github/GustavoNicodemos/Algoritmo_analise_resultado/blob/main/Algoritmo_Analise_GUV_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxlsb
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

# ==============================================================
# Classe principal para análise contábil automatizada
# ==============================================================
class AccountingAnalyzer:
    """
    Classe para análise contábil automatizada:
    - Detecta anomalias, agrupa clusters e gera explicações.
    - Consolida dados por "Group_Account" e calcula variações.
    """
    def __init__(self):
        # Modelos utilizados na análise
        self.anomaly_detector = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
        self.clusterer = MiniBatchKMeans(n_clusters=5, random_state=42)
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.classifier = xgb.XGBClassifier()
        self.history = pd.DataFrame()  # Armazena dados já analisados
        self.treinou = False  # Indica se o modelo foi treinado

    def fit(self, historical_data):
        """
        Treina o modelo de detecção de anomalias com base em dados históricos.
        """
        features = self._preprocess(historical_data)
        self.anomaly_detector.fit(features)
        self.treinou = True
        print("✅ Modelo treinado com sucesso com base histórica.")

    def analyze(self, new_data):
        """Analisa os novos dados com o modelo previamente treinado."""
        if not self.treinou:
            raise ValueError("❌ O modelo precisa ser treinado antes da análise. Use o método .fit() com dados históricos.")

        # Remove duplicados com base no histórico já analisado
        new_data = self._check_history(new_data)

        # Pré-processamento (mesmas transformações do treino)
        features = self._preprocess(new_data)

        # Detectar anomalias
        new_data['anomaly'] = self.anomaly_detector.fit_predict(features)

        # Agrupar em clusters
        new_data['cluster'] = self.clusterer.fit_predict(features)

        # Calcular variações entre períodos
        new_data = self._calculate_variations(new_data)

        # Gerar explicações automáticas
        new_data['explanation'] = self._generate_explanations(new_data)

        # Consolidar por "Group_Account" e ordenar como DRE
        consolidated_data = self._consolidate_data(new_data)

        # Atualiza o histórico
        self._update_history(new_data)

        # Retorna relatório consolidado
        return {
            'anomalias_df': new_data[new_data['anomaly'] == -1],
            'consolidated_df': consolidated_data,
            'explicacoes_df': new_data[['Group_Account', 'explanation']]
        }

    def _check_history(self, data):
        """Remove dados já analisados anteriormente."""
        if self.history.empty:
            return data
        return data[~data.index.isin(self.history.index)]

    def _preprocess(self, data):
        """Codifica dados categóricos e prepara os dados numéricos."""
        cat_data = self.encoder.fit_transform(data[['Cost_Center', 'Group_Account']])
        num_data = data[['Amount_in_LC']].values
        return np.hstack([num_data, cat_data])

    def _calculate_variations(self, data):
        """Calcula variações absolutas e percentuais entre períodos."""
        data['previous_amount'] = data.groupby('Group_Account')['Amount_in_LC'].shift(1)
        data['absolute_variation'] = data['Amount_in_LC'] - data['previous_amount']
        data['percent_variation'] = (data['absolute_variation'] / data['previous_amount']) * 100
        return data

    def _generate_explanations(self, data):
        """Gera justificativas automáticas para anomalias e variações."""
        explanations = []
        for _, row in data.iterrows():
            if row['anomaly'] == -1:
                explanations.append(f"Anomalia detectada em {row['Group_Account']}.")
            elif pd.notnull(row['absolute_variation']) and abs(row['percent_variation']) > 10:
                explanations.append(f"Variação significativa de {row['percent_variation']:.2f}% em {row['Group_Account']}.")
            else:
                explanations.append("Sem anomalias ou variações significativas.")
        return explanations

    def _consolidate_data(self, data):
        """Consolida os dados por 'Group_Account' e ordena como DRE."""
        consolidated = data.groupby('Group_Account').agg({
            'Amount_in_LC': 'sum',
            'absolute_variation': 'sum',
            'percent_variation': 'mean'
        }).reset_index()
        return consolidated.sort_values(by='Group_Account')

    def _update_history(self, data):
        """Salva os dados já analisados para evitar repetições futuras."""
        self.history = pd.concat([self.history, data], ignore_index=True)

In [4]:
# ==============================================================
# Execução do algoritmo
# ==============================================================
!pip install pyxlsb
if __name__ == "__main__":
    # Carregar dados
    base_dados = pd.read_excel('Dados_GUV.xlsb')
    dados_historicos = pd.read_excel('Dados_historicos_GUV.xlsb')

    # Instanciar e treinar o analisador
    analisador = AccountingAnalyzer()
    analisador.fit(dados_historicos)

    # Analisar novos dados
    relatorio = analisador.analyze(base_dados)

    # Salvar outputs em Excel
    relatorio['anomalias_df'].to_excel('anomalias_df.xlsx', index=False)
    relatorio['consolidated_df'].to_excel('consolidated_df.xlsx', index=False)
    relatorio['explicacoes_df'].to_excel('explicacoes_df.xlsx', index=False)

    print("✅ Relatórios gerados com sucesso!")

✅ Modelo treinado com sucesso com base histórica.
✅ Relatórios gerados com sucesso!
